In [65]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [66]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    dim = vec.shape[0]
    
    y = np.zeros(dim)
    y[0] = np.linalg.norm(vec)
    
    u = (vec - y) / np.linalg.norm(vec - y)
    H = np.eye(dim) - np.sign(vec[0]) * 2 * np.outer(u, u)
    
    return y, H

Test your function using tests below:

In [67]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v, atol = 1e-10)
assert_allclose(np.dot(h, v), v1, atol = 1e-10)

In [68]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [69]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    H = np.eye(m)
    R = a1.copy()
    for j in range(n):
        h_j = np.eye(m)
        vec = R[j:, j]
        h = householder(vec)[1]
        h_j[j:, j:] = h
        H = H @ h_j
        R = h_j @ R
    Q = H.T
    return Q, R

In [70]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [71]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=1e-10

Mismatch: 36%
Max absolute difference: 5.00965156
Max relative difference: inf
 x: array([[ 1.      , -0.      , -0.      ,  0.      ,  0.      ],
       [-0.      ,  1.      ,  0.      , -0.      ,  0.      ],
       [-0.      ,  0.      ,  6.009652, -2.308664,  3.106553],...
 y: array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],...

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [72]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [73]:
print(q, '\n')
print(qq, '\n')
print(r, '\n')
print(rr)

[[ 0.13665049  0.56035895  0.19725922  0.62498418  0.48765568]
 [ 0.53601299  0.0935397   0.65948912 -0.50418303  0.12171264]
 [ 0.5563824   1.79687828 -0.89581005 -0.53115059 -1.17759164]
 [ 0.47012864 -1.05543734  0.36580278  0.76961945 -0.05327017]
 [ 0.70771899  0.81843052 -1.17389138 -0.60450035  0.11081392]] 

[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.82705917]
 [ 0.         -0.          0.63442624]
 [ 0.          0.         -0.85368798]] 

[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.     

*Enter your explanation here* (10% of the total grade, peer-graded)

Разложения не совпадают, поскольку в общем случай QR-разложение не единственно.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [74]:
def qr_decomp_modify(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    R = a1.copy()
    A = a1.copy()
    ref_vec = []
    for j in range(n):
        vec = R[j:, j]
        dim = vec.shape[0]
        y = np.zeros(dim)
        y[0] = np.linalg.norm(vec)
        
        u = (vec - y) / np.linalg.norm(vec - y)
        ref_vec.append(u)

        R[j:, j:] -= 2 * np.outer(u, u @ R[j:, j:])
        
    return R, ref_vec

In [75]:
def q_rec(ref_vec, a):
    m, n = a.shape
    q = np.eye(m)
    for j in range(n):
        q[n-j-1:, n-j-1:] -= 2 * np.outer(ref_vec[n-j-1], ref_vec[n-j-1] @ q[n-j-1:, n-j-1:]) 
    return q

In [76]:
q = q_rec(ref_vec, a)

assert_allclose(q.T @ q, np.eye(5), atol=1e-10)
assert_allclose(q.T @ a, R, atol=1e-10)